# Extract PICO instrument parameters

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from https://arxiv.org/abs/1902.10541",
]

In [ ]:
import numpy as np
from astropy.io import fits

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
band_beam = """BandCenter	Beam
21	38.4
25	32
30	28.3
36	23.6
43	22.2
52	18.4
62	12.8
75	10.7
90	9.5
108	7.9
129	7.4
155	6.2
186	4.3
223	3.6
268	3.2
321	2.6
385	2.5
462	2.1
555	1.5
666	1.3
799	1.1
"""

In [ ]:
from pathlib import Path

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
table = QTable(
    names=[
        "telescope",
        "band",
        "center_frequency",
        "fwhm",
        "nside",
        "bandpass_file",
    ],
    dtype=[str, str, float, float, int, str],
    units=[None, None, u.GHz, u.arcmin, None, None],
)

In [ ]:
import io
import requests
import numpy as np

# Parse band_beam and add to table

band_beam_data = np.genfromtxt(
    io.StringIO(band_beam), names=True, dtype=None, encoding="utf-8"
)

for row in band_beam_data:
    table.add_row(
        OrderedDict(
            telescope="PICO",
            band=str(row["BandCenter"]),
            center_frequency=row["BandCenter"] * u.GHz,
            fwhm=row["Beam"] * u.arcmin,
            nside=4096,
            bandpass_file=f"bandpass_{row['BandCenter']}.tbl",
        )
    )

In [ ]:
table

In [ ]:
table.meta["comments"] = ipac_warning
table.write(f"instrument_model.tbl", format="ascii.ipac", overwrite=True)

## Create bandpass files

In [ ]:
for row in table:
    band = row["band"]
    center_freq = row["center_frequency"]
    bandwidth = 0.25 * center_freq
    low_freq = center_freq - bandwidth / 2
    high_freq = center_freq + bandwidth / 2

    freq = np.linspace(low_freq.value, high_freq.value, 10)
    weight = freq ** (-2)
    weight /= weight.max()

    bandpass_table = QTable(
        dict(bandpass_frequency=freq * u.GHz, bandpass_weight=weight)
    )
    bandpass_table.meta["comments"] = ipac_warning
    bandpass_table.write(f"bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
plt.figure(figsize=(10, 6))

for row in table:
    band = row["band"]
    bandpass_table = QTable.read(f"bandpass_{band}.tbl", format="ascii.ipac")
    plt.plot(
        bandpass_table["bandpass_frequency"],
        bandpass_table["bandpass_weight"],
        label=f"Band {band} GHz",
    )

plt.xlabel("Frequency (GHz)")
plt.ylabel("Weight")
plt.title("Bandpasses")
plt.legend()
plt.ylim(0, 1)
plt.grid(True)
plt.show()

# Normalized bandpasses

If we normalize them to the center frequency, all bandpasses are the same

In [ ]:
plt.figure(figsize=(10, 6))

for row in table:
    band = row["band"]
    bandpass_table = QTable.read(f"bandpass_{band}.tbl", format="ascii.ipac")
    center_freq = row["center_frequency"]
    normalized_freq = bandpass_table["bandpass_frequency"] / center_freq
    plt.plot(
        normalized_freq, bandpass_table["bandpass_weight"], label=f"Band {band} GHz"
    )

plt.xlabel("Normalized Frequency")
plt.ylabel("Weight")
plt.title("Normalized Bandpasses")
plt.legend()
plt.ylim(0, 1)
plt.grid(True)
plt.show()